# Housing Regression Maching Learning Project

### Imports (Data and Packages)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from data_preprocessing_final import cleaning, dummify, scale_data, ord_encoding, initiate_data
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor

%matplotlib inline 

In [2]:
housing = pd.read_csv('./data/Ames_Housing_Price_Data.csv', index_col=0)
train_data_linear, train_target_linear, test_data_linear, test_target_linear, train_data_tree, train_target_tree, test_data_tree, test_target_tree = initiate_data(housing)


In [12]:
pickle = pd.read_pickle('lasso_model.pkl')


In [16]:
test_data_linear.drop(['PID'], axis =1, inplace = True)

In [17]:
pickle.score(test_data_linear, test_target_linear)

0.9373935753763666

### Lasso Regression

In [3]:
## Working on Lasso Regression
from sklearn.linear_model import Lasso
lasso = Lasso()

In [4]:
alphaRange = np.linspace(0,0.0005,5)
params = {'alpha': alphaRange, 'max_iter' : [10000]}

In [5]:
gs = GridSearchCV(lasso, params, scoring = 'r2', cv=5)
gs.fit(train_data_linear, train_target_linear)

C:\Users\jackc\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\jackc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\jackc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.073831279823459, tolerance: 0.02087087596564399
  model = cd_fast.enet_coordinate_descent(
C:\Users\jackc\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimato

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': array([0.      , 0.000125, 0.00025 , 0.000375, 0.0005  ]),
                         'max_iter': [10000]},
             scoring='r2')

In [6]:
gs.best_params_

{'alpha': 0.000125, 'max_iter': 10000}

In [7]:
gs.score(train_data_linear, train_target_linear)

0.9538218820797553

In [8]:
gs.score(test_data_linear, test_target_linear)


0.9266089895873977

In [9]:
predict = gs.predict(test_data_linear)
predict = pd.DataFrame(predict)
predict.columns = ['Prediction']

In [ ]:
gs.best_estimator_.fit

<bound method ElasticNet.fit of Lasso(alpha=0.000125, max_iter=10000)>

In [ ]:
features = sorted(list(zip(gs.best_estimator_.coef_, train_data_linear.columns)), key = lambda t: t[0], reverse=True)

In [ ]:
dict = {}
for i in range(len(features)):
    dict[features[i][1]] = features[i][0]



## Establish Connection with Tableau


In [29]:
import tabpy_client
from tabpy.tabpy_tools.client import Client


def predict_price(_arg1,_arg2,_arg3, _arg4):
    scaler = MinMaxScaler()
    test_nums = housing[['TotalLivArea', 'LotArea','GarageArea', 'YearBuilt']]
    scaler.fit(test_nums)


    row = train_data_linear[train_data_linear['PID'] == 1]
    row = row.assign(TotalLivArea = _arg1)
    row = row.assign(LotArea = _arg2)
    row = row.assign(GarageArea = _arg3)
    row = row.assign(YearBuilt = _arg4)

    row_scale = row[['TotalLivArea', 'LotArea','GarageArea', 'YearBuilt']]
    row.drop(columns = row_scale.columns, inplace = True, axis=1)
    row_scaled = scaler.transform(row_scale)
    row_scaled = pd.DataFrame(row_scaled, columns = row_scale.columns)
    row_scaled['PID'] = 1
    row = pd.merge(row, row_scaled, on = 'PID')
    row.drop(['PID'], axis =1, inplace =True)
    price = pickle.predict(row)
    price = int(np.exp(price))
    return price

client = tabpy_client.Client('http://localhost:9004/')
client.deploy('predict_price', predict_price, 'predicts price based on inputs', override = True)




In [ ]:

    housing = pd.read_csv('./data/Ames_Housing_Price_Data.csv', index_col=0)

    # _arg1 = 'OldTown'
    row = housing[housing['PID'] == 1].copy()
    row['Neighborhood'] = _arg1

    housing.drop(housing[housing['PID'] == 1].index, inplace=True)
    housing = housing.append(row)






    #### Clean Data #################################################

    ## LotFrontage -- replace with mean of the column
    housing['LotFrontage'].fillna(value = housing['LotFrontage'].mean(), inplace = True)
    housing['GarageYrBlt'].fillna(value = housing['GarageYrBlt'].mean(), inplace=True)

    ## Replace Nulls where NA just means that the house does not have the feature

    ## Basement metrics
    housing['BsmtQual'].fillna(value = 'No_Basement', inplace=True)
    housing['BsmtCond'].fillna(value = 'No_Basement', inplace=True)
    housing['BsmtExposure'].fillna(value = 'No_Basement', inplace=True)
    housing['BsmtFinType1'].fillna(value = 'No_Basement', inplace=True)
    housing['BsmtFinType2'].fillna(value = 'No_Basement', inplace=True)

    ## Garage Type
    housing['GarageType'].fillna(value = 'No_Garage', inplace=True)

    housing['GarageFinish'].fillna(value = 'No_Garage', inplace=True)
    housing['GarageQual'].fillna(value = 'No_Garage', inplace=True)
    housing['GarageCond'].fillna(value = 'No_Garage', inplace=True)

    ## Replace other nulls where null just means the feature is not there
    housing['FireplaceQu'].fillna(value = 'No_Fireplace', inplace=True)
    housing['PoolQC'].fillna(value = 'No_Pool', inplace=True)
    housing['Fence'].fillna(value = 'No_Fence', inplace=True)
    housing['MiscFeature'].fillna(value = 'No_Misc', inplace=True)
    housing['Alley'].fillna(value = 'No_alley', inplace=True)

    housing['MasVnrType'].fillna(value = 'None', inplace=True)


    ##----------------------------------------------------------------##

    ##Replacing nulls with 0s

    housing['BsmtFinSF1'].fillna(value = 0, inplace=True)
    housing['BsmtFinSF2'].fillna(value = 0, inplace=True)
    housing['BsmtUnfSF'].fillna(value = 0, inplace=True)
    housing['TotalBsmtSF'].fillna(value = 0, inplace=True)
    housing['MasVnrArea'].fillna(value = 0, inplace=True)
    housing['BsmtFullBath'].fillna(value = 0, inplace=True)
    housing['BsmtHalfBath'].fillna(value = 0, inplace=True)
    housing['GarageCars'].fillna(value = 0, inplace=True)
    housing['GarageArea'].fillna(value = 0, inplace=True)


    ## At this point, only one null value remains in the "Electric Column". We will just remove that one row
    housing.dropna(axis = 0, inplace = True)


    ## Create new variables


    ## Add a total baths feature and remove the original columns
    housing['TotalBath'] = housing['FullBath'] + (housing['HalfBath']*0.5) + housing['BsmtFullBath'] + (housing['BsmtHalfBath']*0.5)
    baths_drop = ['HalfBath', 'FullBath', 'BsmtFullBath', 'BsmtHalfBath']
    housing.drop(columns= baths_drop, inplace=True, axis =1)

    ## Add ratio for unfinished basement space -- Make sure this is always before "TotalLivArea" is created or columns will be dropped
    housing['Bsmt_Unfin_Ratio'] = housing['BsmtUnfSF'] / housing['TotalBsmtSF']
    housing.drop(columns = 'BsmtUnfSF', axis = 1, inplace = True)

    ## Add a total living area feature and remove original columns
    housing['TotalLivArea'] = housing['GrLivArea'] + housing['TotalBsmtSF']
    liv_drop = ['GrLivArea', 'TotalBsmtSF']
    housing.drop(columns = liv_drop, axis = 1, inplace = True)

    ## Remove bedrooms from above ground total rooms to avoid multicollinearity
    housing['TotRmsAbvGrd'] = housing['TotRmsAbvGrd'] - housing['BedroomAbvGr']


    ## Remove unnecessary columns
    cols_to_drop = [
        '1stFlrSF',
        '2ndFlrSF',
        'BsmtFinSF1',
        'BsmtFinSF2',
        'Street',
        'Alley',
        'Utilities',
        'Condition2',
        'RoofMatl',
        'Heating',
        'Electrical',
        'LowQualFinSF',
        'KitchenAbvGr',
        'GarageCars',
        'GarageCond',
        'PoolArea',
        'PoolQC',
        'MiscFeature',
        'MiscVal',
        'SaleType']

    housing.drop(columns= cols_to_drop, axis=1, inplace=True)

    # Grouping all the different irregular lotshapes together as 'IR'
    housing.loc[housing.LotShape == 'IR1', 'LotShape'] = 'IR'
    housing.loc[housing.LotShape == 'IR2', 'LotShape'] = 'IR'
    housing.loc[housing.LotShape == 'IR3', 'LotShape'] = 'IR'

    # Grouping all the rare roofstyles together as 'Other'
    housing.loc[housing.RoofStyle == 'Gambrel', 'RoofStyle'] = 'Other'
    housing.loc[housing.RoofStyle == 'Flat', 'RoofStyle'] = 'Other'
    housing.loc[housing.RoofStyle == 'Mansard', 'RoofStyle'] = 'Other'
    housing.loc[housing.RoofStyle == 'Shed', 'RoofStyle'] = 'Other'

    housing['Bsmt_Unfin_Ratio'].fillna(value = 0, inplace = True)

    housing = housing[~housing.PID.isin([904300150, 535383070, 905426030, 528142130])]

    housing = housing[housing['SaleCondition'] == 'Normal']

    housing = housing.drop_duplicates(subset=['PID'], keep='first', ignore_index=False)

    housing = housing.reset_index() #duplicated index values in csv need to reset
    housing = housing.drop('index', axis = 1) # drop original index with duplicates
    ## Grab objects from dataframe
    category = housing.select_dtypes('object')

    ##Grab data that is numerical but should be treated as object (i.e. should be dummified)
    ## Merge it back into category to be dummified
    housing_num2cat = housing[['MSSubClass', 'OverallQual', 'OverallCond', \
                                'MoSold', 'YrSold']]

    category = pd.concat([category, housing_num2cat.astype(str)], axis = 1)

    ## Dummify variables
    cat_dum = pd.get_dummies(category, drop_first = True)

    ## Drop objects out of dataframe
    drop = list(category.columns)
    for column in housing_num2cat.columns:
        drop.append(column)
    housing.drop(columns = drop, axis = 1, inplace = True)

    ## Merge dummified back into original dataframe
    housing = pd.concat([housing, cat_dum], axis = 1)


    ### Dummifying ###############################################################


        ## Train Test Split

    train_data_linear, test_data_linear = train_test_split(housing, test_size=0.2, random_state = 0)
    train_data_linear = train_data_linear.copy()
    test_data_linear = test_data_linear.copy()


    scale_trainer = train_data_linear.copy()
    scale_trainer = scale_trainer.reset_index() #duplicated index values in csv need to reset
    scale_trainer = scale_trainer.drop('index', axis = 1) # drop original index with duplicates
    scale_trainer_num = scale_trainer.select_dtypes(['int64', 'float64']) # Select numeric data types
    scale_trainer_num = scale_trainer_num.drop(['PID', 'SalePrice'], axis = 1) ## Drop PID and saleprice since they should not be scaled

    ## Set up and train the scaler
    scaler = MinMaxScaler()
    scaler.fit(scale_trainer_num)

    housing = housing.reset_index() #duplicated index values in csv need to reset
    housing = housing.drop('index', axis = 1) # drop original index with duplicates
    dataframe_num = housing[['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
                            'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
                            'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
                            'ScreenPorch', 'Bsmt_Unfin_Ratio', 'TotalLivArea', 'TotalBath']] # Select columns that were used to train the scaler

    ## Drop the original columns from the main dataframe
    housing.drop(columns = dataframe_num.columns, axis = 1, inplace =True)

    ## Scale the new columns and make dataframe
    num_scaled = scaler.transform(dataframe_num)
    dataframe_num_scaled = pd.DataFrame(num_scaled, columns = dataframe_num.columns)

    ## Merge back into old dataframe
    housing = pd.concat([housing, dataframe_num_scaled], axis=1)

    ## Separate out target
    target = np.log(housing['SalePrice'])
    housing.drop('SalePrice', axis=1, inplace = True)



    ## Set up linear training and testing data
    # train_data_linear, train_target_linear = scale_data(train_data_linear, scaler)
    # test_data_linear, test_target_linear = scale_data(test_data_linear, scaler)




    ## Set up train test data
    ## note, random seed is set so that PID == 1 will always be in test daata
    # train_data_linear, train_target_linear, test_data_linear, test_target_linear, train_data_tree, train_target_tree, test_data_tree, test_target_tree = initiate_data(housing)

    ## use the model to predict on the test set and then pull out where PID == 1

    train_data_linear = train_data_linear[train_data_linear['PID'] == 1]
    train_data_linear.drop(['PID'] , axis =1, inplace =True)
    predict = pickle.predict(train_data_linear)
    predict = pd.DataFrame(predict)
    predict.columns = ['Prediction']
    z = np.exp(predict['Prediction'])

In [150]:

def predict_for_tableau(_arg1):
    from data_preprocessing_final import initiate_data
    housing = pd.read_csv('./data/Ames_Housing_Price_Data.csv', index_col=0)

    # _arg1 = 'OldTown'
    row = housing[housing['PID'] == 1].copy()
    row['Neighborhood'] = _arg1

    housing.drop(housing[housing['PID'] == 1].index, inplace=True)
    housing = housing.append(row)

    ## Set up train test data
    ## note, random seed is set so that PID == 1 will always be in test daata
    train_data_linear, train_target_linear, test_data_linear, test_target_linear, train_data_tree, train_target_tree, test_data_tree, test_target_tree = initiate_data(housing)

    ## use the model to predict on the test set and then pull out where PID == 1

    train_data_linear = train_data_linear[train_data_linear['PID'] == 1]
    train_data_linear.drop(['PID'] , axis =1, inplace =True)
    predict = pickle.predict(train_data_linear)
    predict = pd.DataFrame(predict)
    predict.columns = ['Prediction']
    z = np.exp(predict['Prediction'])
    return z.iloc[0]

client = tabpy_client.Client('http://localhost:9004/')
client.deploy('predict_for_tableau', predict_for_tableau, 'predicts price based on inputs', override = True)

In [133]:
client = tabpy_client.Client('http://localhost:9004/')
client.deploy('predict_for_tableau', predict_for_tableau, 'predicts price based on inputs', override = True)


In [152]:

predict_for_tableau('BrkSide')



153712.97072623216

In [26]:
_arg1 = 1000
_arg2 = 2000
_arg3 = 1000
_arg4 = 2005

scaler = MinMaxScaler()
test_nums = housing[['TotalLivArea', 'LotArea','GarageArea', 'YearBuilt']]
scaler.fit(test_nums)

row = train_data_linear[train_data_linear['PID'] == 1]
row = row.assign(TotalLivArea = _arg1)
row = row.assign(LotArea= _arg2)
row = row.assign(GarageArea = _arg3)
row = row.assign(YearBuilt = _arg4)

row_scale = row[['TotalLivArea', 'LotArea','GarageArea', 'YearBuilt']]
row.drop(columns = row_scale.columns, inplace = True, axis=1)
row_scaled = scaler.transform(row_scale)
row_scaled = pd.DataFrame(row_scaled, columns = row_scale.columns)
row_scaled['PID'] = 1
row = pd.merge(row, row_scaled, on = 'PID')

row.drop(['PID'], axis =1, inplace =True)
price = pickle.predict(row)
price = int(np.exp(price))

In [27]:
price

65634

In [75]:
final = pd.concat([predict, test_data_linear, test_target_linear], axis =1)
pd.options.display.float_format = '${:,.2f}'.format
final['SalePrice'] = np.exp(final['SalePrice'])
final['Prediction'] = np.exp(final['Prediction'])
final['Pct_diff'] = ((final['Prediction'] - final['SalePrice'])/ final['SalePrice'])*100
final[['Prediction', 'PID','SalePrice', 'Pct_diff']]


,Prediction,PID,SalePrice,Pct_diff
0,"$204,267.84",907181100,"$192,000.00",$6.39
1,"$159,926.91",531452070,"$145,000.00",$10.29
2,"$157,032.98",910204050,"$123,000.00",$27.67
3,"$271,266.09",924151040,"$285,000.00",$-4.82
4,"$150,359.73",923277040,"$138,000.00",$8.96
...,...,...,...,...
477,"$154,205.68",908152110,"$160,500.00",$-3.92
478,"$130,812.19",902427180,"$110,000.00",$18.92
479,"$270,753.79",528180080,"$282,500.00",$-4.16
480,"$136,196.61",534275170,"$151,500.00",$-10.10


## Random Forrest   

In [13]:
# rf_model = RandomForestRegressor(random_state=0)
# # rf_model.fit(train_data_tree, train_target_tree)


RandomForestRegressor(random_state=0)

In [14]:
# train_preds = rf_model.predict(train_data_tree)
# train_mse = mean_squared_error(train_target_tree, train_preds)
# print('Train set MSE is {:.3f}'.format(train_mse))
# print('Train set R2 is {:.3f}'.format(rf_model.score(train_data_tree, train_target_tree)))

# train_preds_test = rf_model.predict(test_data_tree)
# test_mse = mean_squared_error(test_target_tree, train_preds_test)
# print('Test set MSE is {:.3f}'.format(test_mse))
# print('Test set R2 is {:.3f}'.format(rf_model.score(test_data_tree, test_target_tree)))

Train set MSE is 0.002
Train set R2 is 0.987
Test set MSE is 0.014
Test set R2 is 0.900


In [15]:
# predicted = pd.DataFrame(train_preds_test, columns = ['Prediction'])

In [16]:
# final_tree = pd.concat([test_data_tree, predicted, test_target_tree], axis = 1)

In [17]:
# final_tree['SalePrice'] = np.exp(final_tree['SalePrice'])
# final_tree['Prediction'] = np.exp(final_tree['Prediction'])


In [19]:
# ## Look at which features had the most important coefficients 
# sorted(zip(train_data_tree.columns, rf_model.feature_importances_), key = lambda t: t[1], reverse=True)


[('TotalLivArea', 0.611769695940967),
 ('OverallQual', 0.08686860517170654),
 ('YearBuilt', 0.05478411049723338),
 ('ExterQual', 0.049732995132640796),
 ('GarageArea', 0.033503359751477224),
 ('LotArea', 0.01584924789713022),
 ('YearRemodAdd', 0.014954010733209075),
 ('OverallCond', 0.01378381832177892),
 ('GarageYrBlt', 0.009841980604745044),
 ('Fireplaces', 0.00858066722733795),
 ('Bsmt_Unfin_Ratio', 0.008524948559485133),
 ('KitchenQual', 0.007377373224292001),
 ('PID', 0.0066638062850456125),
 ('LotFrontage', 0.005476215459166589),
 ('Neighborhood', 0.004804537389080459),
 ('BsmtFinType1', 0.004671099787307095),
 ('GarageType', 0.0040362385248738355),
 ('MSSubClass', 0.004009441058954551),
 ('OpenPorchSF', 0.0038353831218833796),
 ('CentralAir', 0.0034308679995132658),
 ('MasVnrArea', 0.003281391509908226),
 ('MSZoning', 0.0030934202532820164),
 ('WoodDeckSF', 0.002615876766571731),
 ('MoSold', 0.0026002048637404837),
 ('EnclosedPorch', 0.0024281681202639863),
 ('BsmtQual', 0.00224